In [1]:
from HeaderP import *


In [37]:
APOGEE=pd.read_pickle('./Data/APOGEE_dr16.pkl')
GALAH=pd.read_pickle('./Data/galah_dr3_gaia_cm.pkl')
Kepler_prot=pd.read_pickle('./Data/Kepler_Periodic.pkl')
Kepler_nonprot=pd.read_pickle('./Data/Kepler_Non_Periodic.pkl')
RAVE=pd.read_csv('./Data/ravedr6.csv')
LAMOST=pd.read_csv('./Data/LAMOST_dr6_v2_LRS.csv',delimiter='|')
TESS=pd.read_pickle('./Data/TESS_target.pkl')



/opt/homebrew/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [39]:
APOGEE['survey']='APOGEE'
APOGEE['ra']=APOGEE['RA']
APOGEE['dec']=APOGEE['DEC']
APOGEE['dtypes']='spectra'


GALAH['survey']='GALAH'
GALAH['ra']=GALAH['ra_x']
GALAH['dec']=GALAH['dec_x']
GALAH['dtypes']='spectra'


Kepler_prot['survey']='Kepler'
Kepler_prot['dtypes']='Prot, lightcurves'

RAVE['survey']='RAVE'
RAVE['color']='orange'
RAVE['dtypes']='spectra'

LAMOST['survey']='LAMOST'
LAMOST['dtypes']='spectra'


TESS['survey']='TESS'
TESS['ra']=TESS['RA']
TESS['dec']=TESS['Dec']
TESS['dtypes']='lightcurves'

In [67]:
SurveyDic={0:APOGEE,1:GALAH,2:Kepler_prot,3:Kepler_nonprot,4:RAVE,5:LAMOST,6:TESS}
SurveyDic_name={0:'APOGEE',1:'GALAH',2:'Kepler_prot',3:'Kepler_nonprot',4:'RAVE',5:'LAMOST',6:'TESS'}


In [90]:
from itertools import combinations

cms=list(combinations(range(7), 2))
for i in range(3,7):
    newlist=list(combinations(range(7), i))
    for j in newlist:
        cms.append(j)


In [91]:
len(cms)

119

In [92]:
def crossmatch(df1,df2,sep=1.2):
    # df1
    c1 = SkyCoord([i*u.deg for i in df1['ra']], [i*u.deg for i in df1['dec']], frame='icrs')
    # apogee
    c2 = SkyCoord([i*u.deg for i in df2['ra']], [i*u.deg for i in df2['dec']], frame='icrs')
    # cross-match
    idx_sdss, d2d_sdss, d3d_sdss = c2.match_to_catalog_sky(c1)
    idx=idx_sdss[d2d_sdss<sep*u.arcsec]
    d2d=d2d_sdss[d2d_sdss<sep*u.arcsec]

    df1df2=df2[d2d_sdss<sep*u.arcsec]
    for i in df1.columns:
        df1df2[i]=df1.iloc[idx][i].values
        
    return df1df2

In [89]:
from tqdm import trange

for i in trange(len(cms)):
    # how many surveys to cm
    cmlen=len(cms[i])
    
    df1df2=crossmatch(SurveyDic[cms[i][0]],SurveyDic[cms[i][1]])
    cmname=SurveyDic_name[cms[i][0]]+'_cm_'+SurveyDic_name[cms[i][1]]
    if len(df1df2)==0:
        continue
    print(cmname)
    if cmlen>2:
        for j in range(1,cmlen-1):
            print(SurveyDic_name[cms[i][j+1]])
            df1df2=crossmatch(df1df2,SurveyDic[cms[i][j+1]])
            cmname=cmname+'_cm_'+SurveyDic_name[cms[i][j+1]]
            print(len(df1df2))
            if len(df1df2)==0:
                break
        if len(df1df2)!=0:
            df1df2.to_pickle('Data/'+cmname+'.pkl')

  0%|          | 0/63 [00:00<?, ?it/s]<ipython-input-80-dbb9872c85d2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1df2[i]=df1.iloc[idx][i].values


APOGEE_cm_GALAH
Kepler_prot


  2%|▏         | 1/63 [00:33<34:49, 33.71s/it]

0
APOGEE_cm_GALAH
Kepler_prot


  3%|▎         | 2/63 [01:07<34:14, 33.68s/it]

...

 97%|█████████▋| 61/63 [1:01:51<01:04, 32.25s/it]

0
APOGEE_cm_Kepler_prot
Kepler_nonprot


 98%|█████████▊| 62/63 [1:02:08<00:27, 27.81s/it]

0


100%|██████████| 63/63 [1:02:25<00:00, 59.45s/it]


In [104]:
# read in data
for i in range(len(cms)):
    # how many surveys to cm
    cm_survey=[]
    cm_survey.append(SurveyDic_name[cms[i][0]])
    cm_survey.append(SurveyDic_name[cms[i][1]])
    cmname=SurveyDic_name[cms[i][0]]+'_cm_'+SurveyDic_name[cms[i][1]]
    cmlen=len(cms[i])
    if cmlen>2:
        for j in range(1,cmlen-1):
            cm_survey.append(SurveyDic_name[cms[i][j+1]])
            cmname=cmname+'_cm_'+SurveyDic_name[cms[i][j+1]]
    
    try:
        df1df2=pd.read_pickle('Data/'+cmname+'.pkl')
        print(cm_survey,',',cmlen,',',len(df1df2))
    except:
        continue

['APOGEE', 'GALAH'] , 2 , 14399
['APOGEE', 'Kepler_prot'] , 2 , 2108
['APOGEE', 'Kepler_nonprot'] , 2 , 2388
['APOGEE', 'RAVE'] , 2 , 6858
['APOGEE', 'LAMOST'] , 2 , 225637
['APOGEE', 'TESS'] , 2 , 33311
['GALAH', 'RAVE'] , 2 , 35844
['GALAH', 'LAMOST'] , 2 , 84370
['GALAH', 'TESS'] , 2 , 46265
['Kepler_prot', 'LAMOST'] , 2 , 21897
['Kepler_prot', 'TESS'] , 2 , 459
['Kepler_nonprot', 'LAMOST'] , 2 , 51323
['Kepler_nonprot', 'TESS'] , 2 , 234
['RAVE', 'LAMOST'] , 2 , 15819
['RAVE', 'TESS'] , 2 , 101767
['LAMOST', 'TESS'] , 2 , 34766
['APOGEE', 'GALAH', 'RAVE'] , 3 , 1176
['APOGEE', 'GALAH', 'LAMOST'] , 3 , 5998
['APOGEE', 'GALAH', 'TESS'] , 3 , 5953
['APOGEE', 'Kepler_prot', 'LAMOST'] , 3 , 3396
['APOGEE', 'Kepler_prot', 'TESS'] , 3 , 218
['APOGEE', 'Kepler_nonprot', 'LAMOST'] , 3 , 3850
['APOGEE', 'Kepler_nonprot', 'TESS'] , 3 , 87
['APOGEE', 'RAVE', 'LAMOST'] , 3 , 833
['APOGEE', 'RAVE', 'TESS'] , 3 , 5841
['APOGEE', 'LAMOST', 'TESS'] , 3 , 3910
['GALAH', 'RAVE', 'LAMOST'] , 3 , 799
[

In [98]:
cms[i]

(0, 1)